In [3]:
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from numba import jit
import ipywidgets as wid
from ipywidgets import interactive

RY=13.60566 #x*RY: Rydberg -> ev
ang=1.88972 #X/ang: raio de bohr -> angstroms

def diagg(boo):
    global x,psi,prob,eig,V,L
    
    @jit
    def diagH(H,V):
        H=H+V
        eig, psi=np.linalg.eigh(H) #Resolve o sistema e atribui-se: eig=autovalores, psi=autovetores
        prob=np.abs(psi)**2 #Atribui a densidade de probabilidade |Psi|² a variável prob
        return H,V,psi,prob,eig
    
    N=x.size #Armazena o tamanho do vetor posição e o comprimento do espaço que ele representa
    
    
    a=- 1/(dx**2) #Define 'a' por conveniência
    H=sparse.diags([a,-2*a,a],[-1,0,1],shape=(N,N),dtype=np.float64).toarray() #Cria a matriz Hamiltoniano
    H,V,psi,prob,eig=diagH(H,V)
    
    eig=eig*RY #Ryd->eV
    x=x/ang #RBohr->angstroms
    DIAG.disabled=True
    DIAG.description='Diagonalizado!'
    
def syst(xx,N):   
    global x,dx,V
    
    def vetores(xx,N):
        xmin,xmax=xx
        x,dx=np.linspace(xmin*ang,xmax*ang, N, retstep=True,dtype=np.float64) #Define o vetor posição (raio de bohr)
        return x,dx,xmin,xmax
    
    def pot(N):
        V=sparse.diags([0],[0],shape=(N,N),dtype=np.float64).toarray() #Cria uma matriz NxN com o potencial V(x)
        return V
    
    x,dx,xmin,xmax=vetores(xx,N)
    L=(x.max()-x.min())
    V=pot(N)
    info.children[1].value=f"{dx/ang: .2f}"
    info.children[3].value=f"{L/ang: .2f}"
    
    
#    fig1, ax1= plt.subplots(figsize=(12,8))
    
#    ax1.set_xlabel('$x (\AA)$')
#    ax1.set_ylabel('Potencial (eV)')   
#    plt.plot(x,np.diag(V)*RY,'k-',label='$V(x)$')
#    plt.axvline(x=x.min(),color='k',ls='--',label='x mínimo')
#    plt.axvline(x=x.max(),color='k',ls='--',label='x máximo')
#    leg=plt.legend(loc='upper right')
    DIAG.disabled=False
    DIAG.description='Diagonalizar'
    return V,x,dx,xmin,xmax
    
def plotall(NES,limx,wav,xs,ys):
    global V
    #V=np.diag(V,k=0)
    fig, ax= plt.subplots(figsize=(xs,ys))
    nmin,nmax=NES
    
    ax.set_xlim([x.min()-limx, x.max()+ limx])
    ax.set_xlabel('$x (\AA)$')
    ax.set_ylabel('Auto energias (eV)')
    
    if wav==1:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x,prob[:,i]+eig[i], 
                         label=f'$E_{i}={eig[i]:2f}$ eV')
    elif wav==2:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x,np.real(-1*psi[:,i])+eig[i], 
                         label=f'$E_{i}={eig[i]:2f}$ eV')
    elif wav==3:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x,np.real(psi[:,i])+eig[i], 
                         label=f'$E_{i}={eig[i]:2f}$ eV')
    elif wav==4:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x,np.imag(psi[:,i])+eig[i], 
                         label=f'$E_{i}={eig[i]:2f}$ eV')
            
    fig=plt.axvline(x=x.min(),color='k',ls='--')
    fig=plt.axvline(x=x.max(),color='k',ls='--')
    #fig=plt.plot(x,V,'k',label='V($x$)')
    leg=plt.legend(loc='upper right')
    plt.show()

lslider=wid.Layout(width='90%', display='flex');
sty = {'description_width': 'initial'};


DIAG=wid.Button(description='Diagonalizar',
                disabled=True)

SYST=interactive(syst, {'manual': True}, 
                    xx=wid.FloatRangeSlider(
                        value=[-50.0, 50.0],
                        min=-150.0,
                        max=150.0,
                        step=1,
                        continuous_update=False,
                        orientation='horizontal',
                        readout_format='1d',
                        layout=lslider,
                        description=('Domínio de $\Psi(x)$: '),
                        style=sty),
                    N=wid.IntSlider(value=500,
                        min=50,
                        max=5000,
                        step=10,
                        continuous_update=False, 
                        layout=lslider,
                        description=('Número de pontos: '),
                        style=sty))


PLOT=interactive(plotall, {'manual': True}, 
                    NES=wid.IntRangeSlider(
                        value=[0, 3],
                        min=0,
                        max=10,
                        step=1,
                        description='Número de estados:',
                        continuous_update=False,
                        orientation='horizontal',
                        readout_format='.1d',
                        layout=lslider,
                        style=sty),
                    limx=wid.IntSlider(
                        value=20,
                        min=0,
                        max=80,
                        step=1,
                        description='Margens horizontais em $x$:   ',
                        continuous_update=False,
                        layout=lslider,
                        style=sty),
                    wav=wid.Dropdown(
                        options=[('densidades de probabilidades',1),
                                 ('Funções de onda (parte real, +)',2),
                                 ('Funções de onda (parte real, -)',3),
                                 ('Funções de onda (parte imaginária)',4)],
                        value=1,
                        description='Plotar as',
                        continuous_update=False,
                        style=sty),
                    xs=wid.BoundedIntText(
                        value=16,
                        min=0,
                        max=20,
                        step=1,
                        description='Tamanho da figura (X):',
                        continuous_update=False,
                        style=sty),
                    ys=wid.BoundedIntText(
                        value=9,
                        min=0,
                        max=20,
                        step=1,
                        description='Tamanho da figura (Y):',
                        continuous_update=False,
                        style=sty))

items = [wid.Label('Passo $\Delta x (Å)$ ='), wid.Label('dx'), 
            wid.Label('Largura do domínio $(Å)$ ='), wid.Label('L')]
info=wid.GridBox(items, layout=wid.Layout(grid_template_columns="repeat(2, 200px)"))

SYST.children[2].description=('Gerar');
PLOT.children[5].description=('Plotar');
DIAG.on_click(diagg)

uip=wid.VBox([SYST,info,DIAG])

tab=wid.Tab(children=[uip,PLOT])
tab.set_title(0, 'Gerar o Sistema')
tab.set_title(1, 'Funções de Onda')

display(tab)

***
# Como usar

* 1. Execute o código acima.

* 2. Na aba 'Gerar o Sistema' escolha os parâmetros e clique em 'Gerar' para gerar a equação matricial.
    * Domínio de $\Psi(x)$: o intervalo fechado $X=[x_{min},x_{max}]$ onde a função de onda será descrita.
    * Número de pontos: o número de pontos que serão usados para descrever a função de onda.
    
* 3. Clique em 'Diagonalizar' para resolver o sistema matricial. 

* 4. Na aba 'Funções de Onda' escolha os parâmetros do plot e clique em 'Plotar'.

***